In [2]:
import numpy as np
from flask import Flask
from flask_pymongo import PyMongo
from PIL import Image
import io

# Configuración de la aplicación Flask
app = Flask(__name__)
app.config['MONGO_URI'] = 'mongodb://localhost:27017/caltech101db'
mongo = PyMongo(app)

def save_image_as_bytes(image_array):
    # Convertir el array NumPy a imagen PIL
    image_pil = Image.fromarray((image_array * 255).astype('uint8'))

    # Guardar la imagen en un buffer
    buffer = io.BytesIO()
    image_pil.save(buffer, format='PNG')
    return buffer.getvalue()

def populate_db():
    # Conectar a la colección en MongoDB
    images_collection = mongo.db.images

    # Limpiar la colección antes de poblarla
    images_collection.delete_many({})
    
    try:
        # Cargar las imágenes y etiquetas desde el archivo .npz
        data = np.load('caltech101_train_features.npz')
    except Exception as e:
        print(f"Error al cargar el archivo .npz: {e}")
        return
    
    # Verificar si las claves existen en el archivo .npz
    if 'labels' not in data or 'Images' not in data:
        print("El archivo .npz no contiene las claves 'labels' o 'Images'")
        return
    
    train_labels_loaded = data['labels']
    train_images_loaded = data['Images']

    # Verificar que la cantidad de imágenes y etiquetas coincide
    if len(train_images_loaded) != len(train_labels_loaded):
        print("La cantidad de imágenes y etiquetas no coincide")
        return

    # Poblar la base de datos con las imágenes de entrenamiento
    for i in range(len(train_images_loaded)):
        image_array = train_images_loaded[i]
        label = str(train_labels_loaded[i])
        
        # Convertir la imagen a bytes
        image_bytes = save_image_as_bytes(image_array)
        
        # Insertar el documento en MongoDB
        document = {
            'image': image_bytes,
            'label': label,
            'index': i
        }
        
        try:
            images_collection.insert_one(document)
        except Exception as e:
            print(f"Error al insertar el documento en MongoDB: {e}")
            return

    print("Base de datos poblada con éxito")

if __name__ == "__main__":
    with app.app_context():
        populate_db()


Base de datos poblada con éxito


In [3]:
import numpy as np
from pymongo import MongoClient
from PIL import Image
import io

# Conectar a MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['caltech101db']
collection = db['images']

# Cargar los datos desde el archivo NPZ
data = np.load('caltech101_train_features.npz')
npz_images = data['Images']

# Comparar las imágenes en NPZ y MongoDB
for index in range(len(npz_images)):
    # Obtener imagen de NPZ
    npz_image_array = npz_images[index]
    npz_image = Image.fromarray((npz_image_array * 255).astype('uint8'))

    # Obtener imagen de MongoDB
    image_doc = collection.find_one({'index': index})
    if image_doc:
        mongo_image = Image.open(io.BytesIO(image_doc['image']))
        
        # Comparar las dos imágenes
        if np.array_equal(np.array(npz_image), np.array(mongo_image)):
            print(f"Image at index {index} matches")
        else:
            print(f"Image at index {index} does not match")
    else:
        print(f"No image found in MongoDB at index {index}")


Image at index 0 matches
Image at index 1 matches
Image at index 2 matches
Image at index 3 matches
Image at index 4 matches
Image at index 5 matches
Image at index 6 matches
Image at index 7 matches
Image at index 8 matches
Image at index 9 matches
Image at index 10 matches
Image at index 11 matches
Image at index 12 matches
Image at index 13 matches
Image at index 14 matches
Image at index 15 matches
Image at index 16 matches
Image at index 17 matches
Image at index 18 matches
Image at index 19 matches
Image at index 20 matches
Image at index 21 matches
Image at index 22 matches
Image at index 23 matches
Image at index 24 matches
Image at index 25 matches
Image at index 26 matches
Image at index 27 matches
Image at index 28 matches
Image at index 29 matches
Image at index 30 matches
Image at index 31 matches
Image at index 32 matches
Image at index 33 matches
Image at index 34 matches
Image at index 35 matches
Image at index 36 matches
Image at index 37 matches
Image at index 38 matc

In [4]:
image_bytes= save_image_as_bytes(npz_images[0])
# Depuración de tamaño de imagen
print(f"Image size: {len(image_bytes)} bytes")


Image size: 37224 bytes
